   # Hello Numpy with FLARE API

In this notebook, Hello Numpy is run with the FLARE API to execute commands for submitting the job and following along to see the progress.

### 1. Install NVIDIA FLARE

Follow the [Installation](https://nvflare.readthedocs.io/en/main/getting_started.html#installation) instructions to set up an environment that has NVIDIA FLARE installed if you do not have one already. You will need an environment to run a provisioned FL system.

### 2. Provision and Start FL System

In the rest of this example, we assume that 'nvflare provision' has been run in a workspace (set to '/workspace' below, but you can change this to the location you run provision from) to set up a project named `hello-example` with a server and two clients. Feel free to use an existing provisioned NVFLARE project if you have that available, or to try things out, you could set up and start a system in [POC mode](https://nvflare.readthedocs.io/en/main/getting_started.html#setting-up-the-application-environment-in-poc-mode).

Use the 'start.sh' scripts to start the server and clients in separate terminals to start the system.


### 3. Connect to the FL System with the FLARE API

Use `new_secure_session()` to initiate a session connecting to the FL Server with the FLARE API. The necessary arguments are the username of the admin user you are using and the corresponding startup kit location.

In the code example below, we get the `admin_user_dir` by concatenating the workspace root with the default directories that are created if you provision a project with a given project name. You can change the values to what applies to your system if needed.

Note that if debug mode is not enabled, there is no output after initiating a session successfully, so instead we print the output of `get_system_info()`. If you are unable to connect and initiate a session, make sure that your FL Server is running and that the configurations are correct with the right path to the admin startup kit directory.

In [ ]:
import os
from nvflare.fuel.flare_api.flare_api import new_secure_session

project_name = "example_project"
username = "admin@nvidia.com"
workspace_root = "/tmp/nvflare/poc"
admin_user_dir = os.path.join(workspace_root, project_name, "prod_00", username)

sess = new_secure_session(
    username=username,
    startup_kit_location=admin_user_dir
)
print(sess.get_system_info())

### 4. Submit the Job with the FLARE API

With a session successfully connected, you can use `submit_job()` to submit your job. You can change `path_to_example_job` to the location of the job you are submitting (make sure you have exported the job with the line containing `job.export_job()` uncommented in [fedavg_script_runner_hello-numpy.py](fedavg_script_runner_hello-numpy.py)). If your session is not active, go back to the previous step and connect with a session.

With POC command, we link the examples to the following directory ``` /tmp/nvflare/poc/example_project/prod_00/admin@nvidia.com/transfer```

In [ ]:
ls -l /tmp/nvflare/poc/example_project/prod_00/admin@nvidia.com/transfer


In [ ]:
path_to_example_job = "/tmp/nvflare/jobs/job_config/hello-fedavg-numpy"
job_id = sess.submit_job(path_to_example_job)
print(job_id + " was submitted")

### 5. After Submitting the Job

You should be able to see the output in the terminals where you are running your FL Server and Clients when you submitted the job. You can also use `monitor_job()` to follow along and give you updates on the progress until the job is done.

By default, `monitor_job()` only has one required argument, the `job_id` of the job you are waiting for, and the default behavior is to wait until the job is complete before returning a Return Code of `JOB_FINISHED`.

In order to follow along and see a more meaningful result, the following cell contains the `basic_cb_with_print` callback that keeps track of the number of times the callback is run and prints the `job_meta` the first three times and the final time before `monitor_job()` completes with every other call just printing a dot to save output space. This callback improves the output and is just an example of what can be done with additional arguments and the `job_meta` information of the job that is being monitored.

In [ ]:
from nvflare.fuel.flare_api.flare_api import Session, basic_cb_with_print


sess.monitor_job(job_id, cb=basic_cb_with_print, cb_run_counter={"count":0})

### 6. Shutting Down the FL System

As of now, there is no specific FLARE API command for shutting down the FL system, but the FLARE API can use the `do_command()` function of the underlying AdminAPI to submit any commands that the FLARE Console supports including shutdown commands to the clients and server:

In [ ]:
print(sess.api.do_command("shutdown client"))
print(sess.api.do_command("shutdown server"))

sess.close()